# GEE Library Overview

Writing this guide resulted in a small collection of useful high-level helper functions that make using Google Earth Engine simpler. This notebook goes over some of them.

In [ ]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
import numpy as np

import pprint as pp
import ee
ee.Initialize()

from gee_library import *

import pprint as pp

## date_slices

Dealing with imagery from different times can be difficult. Since different images were taken on different days and mosaics can be stiched together from images taken at different times, you usually need to select images taken on a *range* of dates to stich together an entire area. If you want to see an area change over time you have to select different date ranges that 1) provide 100% coverage of the area of interest and 2) do not overlap, since then you would be comparing two mosaics possibly made up of the same images.

The `date_slices` method trys to help out by finding all the dates available for the images in a cetain area and then finds non-overlapping date ranges that result in 100% coverage of the area of interest. The result is a list of date ranges.

The mechanism that this function uses is rather server-intensive and slow. This method should really only be used in the exploratory phase of a project.

In [ ]:
SAR_dataset = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')
SAR_dataset = ee.ImageCollection('USDA/NAIP/DOQQ')

meters = 3000
pixels = 200
resolution = meters/pixels


nps_center = [ -121.873925, 36.596853]
laplata = [-76.915988, 38.571057]
tile_bounds = square_centered_at(
    point = laplata,
    half_distance = meters/2
)


monterey_SAR_Collection = SAR_dataset.filterBounds(tile_bounds)



print "Getting date slices..."
date_slice_list = date_slices(monterey_SAR_Collection.select(['R', 'G', 'B']), tile_bounds)


print "Done.", len(date_slice_list), "slices found. Getting imagery for each slice..."
for start_date, end_date in date_slice_list:
    
    print timestamp_to_datetime(start_date.getInfo()['value']), "through", timestamp_to_datetime(end_date.getInfo()['value']),":"
    
    filtered_collection = monterey_SAR_Collection.filter(ee.Filter.date(start_date, end_date))

    # Request imagery
    tiles = img_at_region(geCollection=filtered_collection,
                          resolution=resolution,
                          bands=['R', 'G', 'B'],
                          geo_bounds=tile_bounds)
    # resize img to requested size
    np_band_array = [scipy.misc.imresize(tiles[b], (pixels, pixels)) for b in ['R', 'G', 'B']]
    
    # and stack the images in a matrix
    img = np.dstack(np_band_array)
    
    # Display the image
    plt.imshow(img); plt.show()
